# Importing Neccessary Packages

In [1]:
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Using URL 

In [2]:
url1 = 'https://www.adzuna.in/search?adv=1&qwd=Artificial%20Intelligence&f=3&w=India&pp=25&sb=date&sd=down'

# Path to Microsoft Edge WebDriver

In [3]:
edge_driver_path = r"C:\Users\bshiv\Downloads\edgedriver_win64\msedgedriver.exe"

In [4]:
# Creating a Service object for Edge WebDriver
service = Service(edge_driver_path)

# Initializing the Edge WebDriver

In [6]:
driver = webdriver.Edge(service=service)

# Setting implicit wait and loading the URL
driver.implicitly_wait(30)
driver.get(url1)

# Initializing a list to store job details

In [7]:
job_listings = []

# Extracting the total number of jobs

In [8]:
total_jobs_text = driver.find_element(By.CLASS_NAME, 'font-semibold.inline.md\\:font-normal.mr-2.md\\:text-2xl').text
total_jobs = int(re.findall(r'\d+', total_jobs_text)[0]) 
total_jobs

370

# Scraping the URL

In [9]:
while len(job_listings) < total_jobs:
    # Scrolling to the bottom of the page to load more content 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  

    # Extracting job listings
    jobs = driver.find_elements(By.CSS_SELECTOR, "article")  

    for job in jobs:
        try:
            title = job.find_element(By.XPATH, ".//h2/a").text
            company = job.find_element(By.CLASS_NAME, "ui-company").text
            location = job.find_element(By.CLASS_NAME, "ui-location").text
            description = job.find_element(By.CLASS_NAME, "max-snippet-height").text
            link = job.find_element(By.XPATH, ".//h2/a").get_attribute("href")

            # Appending the job details to the list
            job_listings.append({
                "Job Title": title,
                "Company Name": company,
                "Location": location,
                "Job Description": description,
                "Application Link": link
            })
        except Exception as e:
            print(f"Error extracting job details: {e}")

    # Clicking the "next" link to go to the next page
    try:
        # Checking if the "next" link is present
        next_links = driver.find_elements(By.XPATH, "//a[contains(text(), 'next')]")
        if next_links:
            next_link = next_links[0]
            driver.execute_script("arguments[0].click();", next_link)
            time.sleep(3)  
        else:
            print("No more pages to navigate.")
            break
    except Exception as e:
        print(f"Error navigating to the next page: {e}")
        break  

No more pages to navigate.


# Converting the job listings to a DataFrame

In [10]:
df_jobs = pd.DataFrame(job_listings)

# Printing the DataFrame
print(df_jobs)


                                             Job Title  \
0                      Senior Software Engineer, AI/ML   
1                                  IT Security Analyst   
2                             ITV System Test Engineer   
3                             Senior Software Engineer   
4           Infrastructure Specialist: Cloud Platforms   
..                                                 ...   
349     Helpdesk Associate - Technical Troubleshooting   
350  TMS Product Owner - Digital Solution Cnslt. Sr...   
351                        Systems Engineering Advisor   
352              Helpdesk Senior Representative - ITIL   
353            Software Development Specialist Advisor   

                           Company Name  \
0                              VERANTOS   
1          NEXUS COGNITIVE TECHNOLOGIES   
2    IND APTIV COMPONENTS INDIA PRIVATE   
3                       THOMSON REUTERS   
4                                   IBM   
..                                  ...   
34